In [1]:
import datetime
from datetime import date

import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine


In [2]:
# database connections
with open('../config_fill.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_aw = config['Adventure_Works']
    config_etl = config['ETL_PRO']

url_aw = f"mssql+pyodbc://@{config_aw['host']}/{config_aw['dbname']}?driver={config_aw['driver'].replace(' ', '+')}&trusted_connection={config_aw['trusted_connection']}"
url_etl = f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}"

aw_engine = create_engine(url_aw)
etl_engine = create_engine(url_etl)

Extraesmos SalesPerson + Employes + Person

In [3]:
query_salesperson = """
SELECT
    sp.BusinessEntityID AS salesperson_id,
    p.FirstName,
    p.LastName,
    e.JobTitle,
    sp.SalesQuota,
    sp.Bonus,
    sp.CommissionPct,
    sp.SalesYTD,
    sp.SalesLastYear
FROM Sales.SalesPerson sp
JOIN HumanResources.Employee e
    ON sp.BusinessEntityID = e.BusinessEntityID
JOIN Person.Person p
    ON sp.BusinessEntityID = p.BusinessEntityID;
"""

df_salesperson = pd.read_sql(query_salesperson, aw_engine)
df_salesperson.head()


,salesperson_id,FirstName,LastName,JobTitle,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear
0,274,Stephen,Jiang,North American Sales Manager,NaN,0.0,0.000,5.596976e+05,0.000000e+00
1,275,Michael,Blythe,Sales Representative,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06
2,276,Linda,Mitchell,Sales Representative,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06
3,277,Jillian,Carson,Sales Representative,250000.0,2500.0,0.015,3.189418e+06,1.997186e+06
4,278,Garrett,Vargas,Sales Representative,250000.0,500.0,0.010,1.453719e+06,1.620277e+06


Transform

In [4]:
df_salesperson = df_salesperson.fillna({
    "SalesQuota": 0,
    "Bonus": 0,
    "CommissionPct": 0,
    "SalesYTD": 0,
    "SalesLastYear": 0,
})


In [5]:
df_salesperson.insert(0, "salesperson_key", range(1, len(df_salesperson) + 1))
df_salesperson.head()


,salesperson_key,salesperson_id,FirstName,LastName,JobTitle,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear
0,1,274,Stephen,Jiang,North American Sales Manager,0.0,0.0,0.000,5.596976e+05,0.000000e+00
1,2,275,Michael,Blythe,Sales Representative,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06
2,3,276,Linda,Mitchell,Sales Representative,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06
3,4,277,Jillian,Carson,Sales Representative,250000.0,2500.0,0.015,3.189418e+06,1.997186e+06
4,5,278,Garrett,Vargas,Sales Representative,250000.0,500.0,0.010,1.453719e+06,1.620277e+06


In [6]:
df_salesperson.rename(columns={
    "FirstName": "first_name",
    "LastName": "last_name",
    "JobTitle": "job_title",
    "SalesQuota": "sales_quota",
    "Bonus": "bonus",
    "CommissionPct": "commission_pct",
    "SalesYTD": "sales_ytd",
    "SalesLastYear": "sales_last_year"
}, inplace=True)


Load

In [8]:
df_salesperson.to_sql(
    "dim_salesperson",
    etl_engine,
    if_exists="replace",
    index=False
)


17